In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

def tabularize_fuel_oil_no2_pricing(pricing_source_file, output_file, years_to_average, cost_col, output_cost_column):
    """ 
    Average the monthly national fuel oil number 2 pricing 
    """
    fuel_oil_no2_pricing = pd.read_csv(pricing_source_file)
    fuel_oil_no2_pricing.fillna(0)
    fuel_oil_no2_pricing["Year"] = pd.to_datetime(fuel_oil_no2_pricing['Week of']).dt.year
    fuel_oil_no2_pricing["Month"] = pd.to_datetime(fuel_oil_no2_pricing['Week of']).dt.month
    fuel_oil_no2_pricing = fuel_oil_no2_pricing[ fuel_oil_no2_pricing["Year"].isin(years_to_average) ]

    output_fuel_oil_no2_pricing = []
    for month in fuel_oil_no2_pricing["Month"].unique(): 
        data = fuel_oil_no2_pricing[ fuel_oil_no2_pricing["Month"] == month ]
        output_fuel_oil_no2_pricing.append({'Month': month, output_cost_column: data[cost_col].mean()})
    
    output_fuel_oil_no2_pricing = pd.DataFrame(output_fuel_oil_no2_pricing)
    output_fuel_oil_no2_pricing = output_fuel_oil_no2_pricing.sort_values(by=['Month'])
    output_fuel_oil_no2_pricing.to_csv(output_file, index=False)

    return None

    
def tabularize_monthly_state_specific_pricing(input_pricing_file, output_pricing_file, cost_column):
    """ 
    Convert from a table with different columns for each state's price, to rows where each row is a month of a state and its price. 
    """

    pricing = pd.read_csv(input_pricing_file)
    # set the column names for the natural gas 
    national_and_cities_columns = set(pricing.columns) - set("Date")  
    dataframes_list = []
    for column in national_and_cities_columns: 

        if column in us_state_to_abbrev.keys(): state = us_state_to_abbrev.get(column)
        else: state = column 

        temp_df = pd.DataFrame()
        temp_df["Month"] = pricing["Month"].to_list()
        temp_df["State"] = state
        temp_df[cost_column] = pricing[column].to_list()
        dataframes_list.append(temp_df)
        del temp_df

    pricing = pd.concat(dataframes_list)
    pricing.to_csv(output_pricing_file, index=False)
    
    return None


def average_pricing_by_states(pricing_source_file, years_to_average, exclude_cols_from_average):
    """
    Avereage EIA fuel pricing data by month for each state / region
    """

    pricing = pd.read_csv(pricing_source_file)
    pricing.fillna(0)
    pricing = pricing[ pricing["Year"].isin(years_to_average) ]

    cols_to_average = list(set(pricing.columns) - set(exclude_cols_from_average))

    print(cols_to_average)
    
    dataframes_list = []
    for month in pricing["Month"].unique(): 
        data = pricing[ pricing["Month"] == month ]
        data = {col: [data[col].mean()] for col in cols_to_average}   # List comprehension way to build a dictionary of the month's average price for each column to average 
        data = {**{"Month": month}, **data}                         # Add the month to the front of the dictionary
        dataframes_list.append(pd.DataFrame(data))
        del data

    pricing = pd.concat(dataframes_list)
    pricing = pricing.sort_values(by=['Month'])

    return pricing


def linearly_interpolate_pricing(xa, xb, month_col, pricing):
    """ 
    Linearly interpolate missing EIA fuel pricing data 
    """
    pricing_array = pricing.to_numpy()

    m = (pricing_array[pricing_array[:,month_col] == xb] - pricing_array[pricing_array[:,month_col] == xa]) 
    m = m / m[month_col][month_col]

    b = m*(-xa) + pricing_array[pricing_array[:,month_col] == xa]

    for x in range(xa+1, xb):
        y = np.zeros(np.shape(m))
        y = m*x + b 
        pricing_array = np.vstack([pricing_array, y])

    pricing = pd.DataFrame(pricing_array, columns = pricing.columns)
    pricing["Month"] = pricing["Month"].astype(int)
    pricing = pricing.sort_values(by=['Month'])
    return pricing


us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

# Electricity

# FRED Electricity Data (city / regional)

In [ ]:
def average_electricity_pricing(electricity_pricing_file, years_to_average):
    """ 
    Average the price of electricity by month for the given years of electricity pricing data and ignore missing entries. 
    """

    electricity_pricing = pd.read_csv(electricity_pricing_file)
    electricity_pricing["DATE"] = pd.to_datetime(electricity_pricing["DATE"])
    electricity_pricing = electricity_pricing[electricity_pricing["DATE"].dt.year.isin(years_to_average)]    

    electricity_pricing = electricity_pricing[ ~electricity_pricing.iloc[:, 1].isin(['.']) ]                # ignore missing values for averaging 
    electricity_pricing.iloc[:,1] = electricity_pricing.iloc[:,1].astype(float)                             # convert price column to float 

    # month_data
    average_monthly_prices = []
    for month in electricity_pricing["DATE"].dt.month.unique():
        month_data = electricity_pricing[ electricity_pricing["DATE"].dt.month.isin([month]) ]
        average_monthly_prices.append(pd.DataFrame({"Month": [month], "Mean(Price Electricity ($/kWh))": [month_data.iloc[:,1].mean()]}))
    average_monthly_prices = pd.concat(average_monthly_prices)
    
    if len(average_monthly_prices['Month'].unique()) < 12:
        raise RuntimeError(f'Error: {electricity_pricing_file} has missing monthly pricing values')

    return average_monthly_prices


base_path = Path("/Volumes/seas-mtcraig/data_sharing/Energy Burdens Under Climate Change/Energy rates/Electricity/")
national_electricity_files = Path(base_path, "US City Average/APU000072610.csv")
city_electricity_files = {
    'Phoenix': Path(base_path, "Phoenix, Mesa, Scottsdale/APUS48A72610.csv"),
    'Los Angeles': Path(base_path, "Los Angeles, Long Beach, Anaheim/APUS49A72610.csv"),
    'San Diego': Path(base_path, "San Diego, Carlsbad/APUS49E72610.csv"),
    'San Francisco': Path(base_path, "San Francisco, Oakland, Hayward/APUS49B72610.csv"),
    'Denver': Path(base_path, "Denver, Aurora, Lakewood/APUS48B72610.csv"),

        'Jacksonville': national_electricity_files,

    'Miami': Path(base_path, "Miami, Fort Lauderdale, West Palm Beach/APUS35B72610.csv"),
    'Chicago': Path(base_path, "Chicago, Naperville, Elgin/APUS23A72610.csv"),

        'Indianapolis': national_electricity_files, 
        'Louisville': national_electricity_files, 

    'Baltimore': Path(base_path, "Baltimore, Columbia, Towson/APUS35E72610.csv"),
    'Detroit': Path(base_path, "APUS23B72610_Detroit.csv"),

        'Duluth': national_electricity_files, 
        'Billings': national_electricity_files,
        'Albuquerque': national_electricity_files,

    'New York': Path(base_path, "New York, New Jersey, Pennsylvania/APUS12A72610.csv"),
    'Cleveland': Path(base_path, "Cleveland, Akron/APUA21072610.csv"),

        'Oklahoma City': national_electricity_files, 
        'Portland': national_electricity_files, 

    'Philadelphia': Path(base_path, "New York, New Jersey, Pennsylvania/APUS12A72610.csv"),

        'Memphis': national_electricity_files, 

    'Dallas': Path(base_path, "APUS37A72610_Dallas.csv"),
    'Houston': Path(base_path, "Houston, The Woodlands, Sugar Land/APUS37B72610.csv"),

        'San Antonio': national_electricity_files,
        'Milwaukee': national_electricity_files,

    # 'Atlanta': Path(base_path, "APUS35C72610_Atlanta_GA.csv"),
    }

output_electricity_pricing_file = Path(base_path, "aggregate_data_By_(City,_Month).csv")


In [ ]:

years_to_average = range(2019, 2020 +1)

pricing_data = []
for city in city_electricity_files.keys():
    electricity_pricing = average_electricity_pricing(city_electricity_files.get(city), years_to_average)
    electricity_pricing['City'] = city
    pricing_data.append(electricity_pricing)

pricing_data = pd.concat(pricing_data)

pricing_data.to_csv(output_electricity_pricing_file, index=False)


# EIA Monthly Electric Price Cleanup (State wide pricing)

In [ ]:
electricity_pricing_file = Path("/Users/camilotoruno/Downloads/Average_retail_price_of_electricity (2).csv")
o_file = Path("/Users/camilotoruno/Downloads/EIA_Average_retail_price_of_electricity.csv")
electricity_pricing = pd.read_csv(electricity_pricing_file)

price_table = []
for j, col in enumerate(electricity_pricing.iloc[:,2:-1].columns):
    odata = pd.DataFrame()
    odata['Region'] = electricity_pricing['Region']
    odata['Month'] = col.split('-')[0]
    electricity_pricing[col] = electricity_pricing[col]
    odata['Price (cents per kelowatthour)'] = electricity_pricing[col]
    # print(odata)
    price_table.append(odata)

price_table = pd.concat(price_table)
average_pricing = price_table.groupby(['Region', 'Month'])['Price (cents per kelowatthour)'].mean()
average_pricing = average_pricing.to_frame().reset_index()

import calendar
month_dictionary = {month: index for index, month in enumerate(calendar.month_abbr) if month}	# create month name to number dictionary - use Month: Month number (e.g. October: 10)
if sum(average_pricing['Month'].isin(list(month_dictionary.keys()))) > 0:
    average_pricing['Month'] = average_pricing['Month'].map(month_dictionary)   			# map the month name to month number 

average_pricing['Region'] = average_pricing['Region'].replace(us_state_to_abbrev) 
print(average_pricing)
average_pricing.to_csv(o_file)


# Natural Gas

In [ ]:
nat_gas_pricing_source_file = Path("/Volumes/seas-mtcraig/data_sharing/Energy Burdens Under Climate Change/Energy rates/Natural gas/NG_PRI_SUM_A_EPG0_PRS_DMCF_M By (Month).csv")
nat_gas_pricing_output_table_file = Path("/Users/camilotoruno/Documents/local_research_data/pricing_data", "natural_gas_pricing.csv")

####### (If not done yet) Convert source pricing table (with some minor excel modifications) to table for merging on results ###############
new_cost_column_name = 'Price of Natural Gas Delivered to Residential Consumers (Dollars per Thousand Cubic Feet)'
tabularize_monthly_state_specific_pricing(nat_gas_pricing_source_file, nat_gas_pricing_output_table_file, new_cost_column_name)     



# Fuel Oil

In [ ]:
fuel_oil_no2_pricing_source_file = Path("/Users/camilotoruno/Documents/local_research_data/pricing_data/Weekly_U.S._Weekly_No._2_Heating_Oil_Residential_Price_clean header.csv")
fuel_oil_no2_pricing_file = Path("/Users/camilotoruno/Documents/local_research_data/pricing_data/Monthly_No._2_Heating_Oil_Residential_Price.csv") 

###### (If not done yet) Convert source pricing table (with some minor excel modifications) to table for merging on results ###############
years_to_average = range(2019, 2020 +1)
cost_col = "Weekly U.S. Weekly No. 2 Heating Oil Residential Price Dollars per Gallon"
output_cost_column = 'Monthly No. 2 Heating Oil Residential Price Dollars per Gallon'
tabularize_fuel_oil_no2_pricing(fuel_oil_no2_pricing_source_file, fuel_oil_no2_pricing_file, years_to_average, cost_col, output_cost_column)
print("If youve just created the table for the first time, linearly interpolate the missing data")
## If you've just created the table for the first time, linearly interpolate the missing data 
## You might be able to use the function i created for linearly interpolating. For the current data which is just for the whole US (12 data points), I did the interpolationg in excel


# Propane
## First Average the pricing data to the monthly level from weekly level 

In [ ]:
pricing_source_file = Path("/Volumes/seas-mtcraig/data_sharing/Energy Burdens Under Climate Change/Energy rates/Propane/weekly_USA/weekly_propane_pricing_USA.csv")
average_propane_pricing_file = Path("/Users/camilotoruno/Documents/local_research_data/pricing_data/Monthly_Propane_Price.csv") 

columns_to_exclude_from_averaging = ["Date",	"Month",	"Year"]

# average_pricing_by_states(columns_to_exclude_from_averaging)
years_to_average = range(2019, 2020 +1)
average_propane_pricing_by_states = average_pricing_by_states(pricing_source_file, years_to_average, columns_to_exclude_from_averaging)
average_propane_pricing_by_states.to_csv(average_propane_pricing_file, index=False)
average_propane_pricing_by_states

In [ ]:
average_propane_pricing_by_states = pd.read_csv(average_propane_pricing_file)
interpolated_propane_pricing_file = Path("/Users/camilotoruno/Documents/local_research_data/pricing_data/Monthly_Propane_Price_Interpolated.csv") 

# xa and xb chosen as Months on either side of the missing months section
xa = 3
xb = 10    
month_col = 0

interpolated_average_propane_pricing_by_states = linearly_interpolate_pricing(xa, xb, month_col, average_propane_pricing_by_states)

import matplotlib.pyplot as plt
plt.plot(interpolated_average_propane_pricing_by_states.iloc[:,0], interpolated_average_propane_pricing_by_states.iloc[:,1], marker='o')
plt.title(f"Monthly Propane Average Price - {years_to_average[0]}-{years_to_average[-1]}")

interpolated_average_propane_pricing_by_states.to_csv(interpolated_propane_pricing_file, index=False)

### Now tabularize the data 
Convert from a table with different columns for each state's price, to rows where each row is a month of a state and its price. 

In [ ]:
new_cost_column_name = 'Monthly U.S. Propane Residential Price (Dollars per Gallon)'
tabularize_monthly_state_specific_pricing(interpolated_propane_pricing_file, interpolated_propane_pricing_file, new_cost_column_name)

# interpolated_propane_pricing = pd.read_csv(interpolated_propane_pricing_file)
# interpolated_propane_pricing